In [ ]:
#convert to csv
import pandas as pd
df = pd.read_excel("/Users/pulkitleuva/Desktop/metamapper/metamapper_24042025_ssg_classification_pl_sr.xlsx")
df.to_csv("../input/MASLD_input_metamapper.csv", index = None)

In [35]:
import pandas as pd
#check valid data
df = pd.read_csv("../input/Metamapper_21072025hypertension_Dyslipidemia_hepbc.csv")
#metamapper_20072025_circumference_bp
#Metamapper_21072025hypertension_Dyslipidemia_hepbc.csv
#MASLD_input_metamapper
print("all data points: ",len(df))
df = df.dropna(subset="E Median (kPa)")
print("points with E Median (kPa) values: ", len(df))
df = df[df["Calibration Status"]=="OK"]
print("points with calibration status=OK: ",len(df))
df = df[df["Valid measurement number"]>=10]
print("points with >10 valid measurements: ",len(df))

df = df.loc[df['CAP Enhanced SD (dB/m)'] < 0.20 * df['CAP Enhanced Mean (dB/m)']]
print("points with CAP enhanced <20%: ",len(df))
df = df[df["consumption_alcohol"]=='No']
print("consumption_alcohol",len(df)) #HEPB_label
df = df[df["HEPB_label"]!='Positive']
print("HEPB_label",len(df))
df = df[df["HEPC_label"]!='Positive']
print("HEPC_label",len(df))
df.to_csv("../input/liver_qc.cvs",index=None)

/var/folders/_q/8bx4f_jd77591946tt1yg1sc0000gn/T/ipykernel_34297/2265269494.py:3: DtypeWarning: Columns (123,124,125,126,127,128,129,130,131,135,136,137,138,139,140,141,142,144,145,146,148,149,150,152,153,154,155,156,157,158,159,161,162,163,164,165,166,171,172,176,178,179,180,181,184,188,189,190,191,192,193,197,229,236,247,250,261,262,264,273,280,281,284,285,288,289,307,308,309,310,311,312,313,314,315,316,317,330,564,584,599,601,648,652,659,660,661,662,668,676,677,678,679,680,681,686,688,725,732,733,734,735,740,747,748,749,750,752,753,754,755,887,892,894,899,900,901,975,984) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../input/Metamapper_21072025hypertension_Dyslipidemia_hepbc.csv")


all data points:  10267
points with E Median (kPa) values:  9996
points with calibration status=OK:  9944
points with >10 valid measurements:  9937
points with CAP enhanced <20%:  9923
consumption_alcohol 7890
HEPB_label 7821
HEPC_label 7792


In [7]:
df = pd.read_csv("../input/liver_qc.cvs")
print(len(df['BMI'] >= 23))

0


In [39]:
import pandas as pd

# Load data
df = pd.read_csv("../input/liver_qc.cvs")

# Apply CAP filter
cap = 248

df = df[df["CAP Enhanced Mean (dB/m)"] > cap].copy()

# Define individual criteria columns
df["BMI_cond"] = (df['BMI'] >= 23).astype(int)
df["Male_waist_94"] = ((df['Gender'] == 'Male') & (df['final_waist'] >= 90)).astype(int) #>=94
df["Male_hdl_40"] = ((df['Gender'] == 'Male') & (df['Serum HDL Cholesterol'] <= 39)).astype(int)
df["Female_waist_80"] = ((df['Gender'] == 'Female') & (df['final_waist'] >= 80)).astype(int)#>= 80
df["Female_hdl_50"] = ((df['Gender'] == 'Female') & (df['Serum HDL Cholesterol'] <= 50)).astype(int)
#Systolic (mmHg)
#Diastolic (mmHg)
df["Hypertension_cond"] = ((df['Systolic'] >= 130) | (df['Diastolic'] >= 85) | (df['medHis_highBP_medicine'] == 'Yes')).astype(int)
df["Triglycerides_cond"] = (df['Serum Triglycerides'] >= 150).astype(int)
#medHis_highSugar
df["Diabetes_cond"] = ((df['Diabetic_final'] == 'Yes') | (df['medHis_highSugar'] == 'Yes') ).astype(int)
df["Prediabetes_cond"] = (df['Prediabetics_final'] == 'Yes').astype(int)
#df["Hypertension_cond"] = (df['Hypertension_final'] == 'Yes').astype(int)
df["Chol_med_cond"] = (df['medHis_highChol_medicine'] == 'Yes').astype(int)

# List of condition columns
condition_cols = [
    "BMI_cond", "Male_waist_94", "Male_hdl_40", "Female_waist_80",
    "Female_hdl_50", "Triglycerides_cond", "Diabetes_cond", "Prediabetes_cond",
    "Hypertension_cond", "Chol_med_cond"
]

# Sum of all conditions per row
df[f'MASLD_{cap}_criteria'] = df[condition_cols].sum(axis=1)
df[f'MASLD_{cap}'] = (df[f'MASLD_{cap}_criteria'] > 0).astype(int)
# Save matrix to CSV
df.to_csv(f"../output/MASLD_matrix_{cap}.csv", index=False)

# Print how many rows have at least one condition met
print("Rows with any condition met:", (df[f'MASLD_{cap}'] > 0).sum())
print("Rows without any condition met:", (df[f'MASLD_{cap}'] == 0).sum())

/var/folders/_q/8bx4f_jd77591946tt1yg1sc0000gn/T/ipykernel_34297/113922195.py:4: DtypeWarning: Columns (123,124,125,127,128,129,130,131,135,136,137,138,139,140,141,142,144,145,146,148,149,150,152,153,154,155,156,157,158,159,161,162,163,164,165,166,171,172,176,178,180,181,184,188,189,190,191,193,229,236,247,250,261,262,264,273,280,281,284,285,288,289,307,308,309,310,311,312,313,314,315,316,317,330,668,676,677,678,679,680,681,686,687,688,892,899,901,942,975,984) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../input/liver_qc.cvs")


Rows with any condition met: 3712
Rows without any condition met: 28


In [65]:
# Load main DataFrame
df = pd.read_csv("../input/liver_qc.cvs")

# Load and trim other DataFrames
df2 = pd.read_csv("../output/MASLD_matrix_248.csv")[["LABID", "MASLD_248"]]
df3 = pd.read_csv("../output/MASLD_matrix_263.csv")[["LABID", "MASLD_263"]]
df4 = pd.read_csv("../output/MASLD_matrix_275.csv")[["LABID", "MASLD_275"]]

# Merge all on LABID (left join)
df = df.merge(df2, on="LABID", how="left")
df = df.merge(df3, on="LABID", how="left")
df = df.merge(df4, on="LABID", how="left")

# Optional: Save merged result
df.to_csv("../output/liver_with_MASLD_flags.csv", index=False)

/var/folders/_q/8bx4f_jd77591946tt1yg1sc0000gn/T/ipykernel_54010/2432371349.py:2: DtypeWarning: Columns (4,6,10,11,12,13,15,17,19,20,21,22,23,24,25,31,37,39,43,49,109,110,112,113,114,115,116,117,120,121,122,123,188,193,203,213,221,224,228,232) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../input/liver_qc.cvs")
/var/folders/_q/8bx4f_jd77591946tt1yg1sc0000gn/T/ipykernel_54010/2432371349.py:5: DtypeWarning: Columns (6,10,11,12,13,15,16,39,43,109,110,112,113,115,117,118,121,122,123,187,188,193,200,215,216,221,222,223,224,226,228,229,231,232) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("../output/MASLD_matrix_248.csv")[["LABID", "MASLD_248"]]
/var/folders/_q/8bx4f_jd77591946tt1yg1sc0000gn/T/ipykernel_54010/2432371349.py:6: DtypeWarning: Columns (6,10,11,12,13,15,16,39,43,112,113,121,122,224) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv("../outp

In [75]:

df1 = pd.read_csv("/Users/pulkitleuva/Desktop/liver/MAFLD/output/liver_with_MAFLD_flags.csv")
df2 =  pd.read_csv("/Users/pulkitleuva/Desktop/liver/MASLD/output/liver_with_MASLD_flags.csv")[["LABID", "MASLD_248","MASLD_263", "MASLD_275"]]
df1 = df1.merge(df2, on="LABID", how="left")
df1.to_csv("../output/liver_with_MASLD_MAFLD_flags.csv", index=False)

/var/folders/_q/8bx4f_jd77591946tt1yg1sc0000gn/T/ipykernel_54010/2865683602.py:1: DtypeWarning: Columns (4,6,10,11,12,13,15,17,19,20,21,22,23,24,25,31,37,39,43,49,109,110,112,113,114,115,116,117,120,121,122,123,188,193,203,213,221,224,228,232) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("/Users/pulkitleuva/Desktop/liver/MAFLD/output/liver_with_MAFLD_flags.csv")
/var/folders/_q/8bx4f_jd77591946tt1yg1sc0000gn/T/ipykernel_54010/2865683602.py:2: DtypeWarning: Columns (4,6,10,11,12,13,15,17,19,20,21,22,23,24,25,31,37,39,43,49,109,110,112,113,114,115,116,117,120,121,122,123,188,193,203,213,221,224,228,232) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 =  pd.read_csv("/Users/pulkitleuva/Desktop/liver/MASLD/output/liver_with_MASLD_flags.csv")[["LABID", "MASLD_248","MASLD_263", "MASLD_275"]]
